In [11]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import MeanSquaredError
from tensorflow.keras.metrics import MeanAbsoluteError
from tensorflow.keras.layers import Dense

data = pd.read_csv('combined_data_aqi.csv')

features = ['co', 'so2', 'nh3', 'pm2_5']
target = 'aqi'

X = data[features]
y = data[target]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

import joblib
joblib.dump(scaler, 'scaler_forecase_aqi.pkl')


['scaler_forecase_aqi.pkl']

In [12]:
model = Sequential([
    Dense(64, activation='relu', input_shape=(len(features),)),
    Dense(32, activation='relu'),
    Dense(16, activation='relu'),
    Dense(7, activation='linear')
])

model.compile(optimizer=Adam(), loss=MeanSquaredError(), metrics=[MeanAbsoluteError()])

history = model.fit(X_train_scaled, y_train, epochs=50, batch_size=32, validation_split=0.2)

model.save('predict_7days_model.h5')


c:\Users\oi\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/50
4138/4138 ━━━━━━━━━━━━━━━━━━━━ 25s 5ms/step - loss: 0.3412 - mean_absolute_error: 0.2733 - val_loss: 0.0439 - val_mean_absolute_error: 0.1132
Epoch 2/50
4138/4138 ━━━━━━━━━━━━━━━━━━━━ 23s 5ms/step - loss: 0.0390 - mean_absolute_error: 0.1032 - val_loss: 0.0353 - val_mean_absolute_error: 0.0928
Epoch 3/50
4138/4138 ━━━━━━━━━━━━━━━━━━━━ 22s 5ms/step - loss: 0.0329 - mean_absolute_error: 0.0881 - val_loss: 0.0328 - val_mean_absolute_error: 0.0794
Epoch 4/50
4138/4138 ━━━━━━━━━━━━━━━━━━━━ 41s 5ms/step - loss: 0.0291 - mean_absolute_error: 0.0768 - val_loss: 0.0289 - val_mean_absolute_error: 0.0732
Epoch 5/50
4138/4138 ━━━━━━━━━━━━━━━━━━━━ 23s 6ms/step - loss: 0.0267 - mean_absolute_error: 0.0693 - val_loss: 0.0276 - val_mean_absolute_error: 0.0697
Epoch 6/50
4138/4138 ━━━━━━━━━━━━━━━━━━━━ 40s 5ms/step - loss: 0.0256 - mean_absolute_error: 0.0656 - val_loss: 0.0255 - val_mean_absolute_error: 0.0679
Epoch 7/50
4138/4138 ━━━━━━━━━━━━━━━━━━━━ 22s 5ms/step - loss: 0.0249 - mean_absol

In [13]:
from tensorflow.keras.models import load_model

model = load_model('predict_7days_model.h5')
scaler = joblib.load('scaler.pkl')

input_data = {
    "co": 5000,
    "so2": 30,
    "nh3": 5,
    "pm2_5": 40
}

input_features = np.array([[input_data['co'], input_data['so2'], input_data['nh3'], input_data['pm2_5']]])
input_scaled = scaler.transform(input_features)

predictions = model.predict(input_scaled)[0]

print("Predicted AQI for the next 7 days:", predictions)


c:\Users\oi\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\base.py:465: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 398ms/step
Predicted AQI for the next 7 days: [3.1392567 3.1001961 3.0880551 3.0972595 3.0929694 3.1242514 3.1243958]
